In [5]:
!pip install fastapi


     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 10.1MB 10.0MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 


In [18]:
pip install transformers==4.3.2

     |████████████████████████████████| 1.8MB 12.7MB/s 
     |████████████████████████████████| 890kB 49.7MB/s 
     |████████████████████████████████| 3.2MB 46.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=964777524998ec80ce6987c9643362655eebdb32b53b0011ed862827fd6ca84a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [45]:
from fastapi import Depends, FastAPI
import pandas as pd
import pickle
import torch
from pydantic import BaseModel
from typing import Dict
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
import json

#B asic setup. To run this however, you will also need Test_Regression.csv and xgb_reg.pkl from the Netflix notebook in the same folder.
# This tutorial gives some basic steps for how to launch using a simple web server like uvicorn: 
# https://fastapi.tiangolo.com/

app = FastAPI()

# # This model would need to be regularly updated as ratings change.
# with open("xgb_reg.pkl", 'rb') as f:
#     clf_loaded = pickle.load(f)

with open("/content/drive/MyDrive/path/to/awesome-name-you-picked/config.json") as json_file:
    config = json.load(json_file)


model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/path/to/awesome-name-you-picked', config=config)
@app.get("/")
async def root():
    return {"Welcome to our cool API!"}


class SentimentRequest(BaseModel):

    text: str

class SentimentResponse(BaseModel):

    probabilities: Dict[str, float]

    sentiment: str

    confidence: float

@app.post("/predict", response_model=SentimentResponse)

def predict(request: SentimentRequest):

    sentiment, confidence, probabilities = model.predict(request.text)

    return SentimentResponse(

        sentiment=sentiment, confidence=confidence, probabilities=probabilities

    )


